# Generate A schedule-puzzle logic-reasoning skill challenge:

In [ ]:
"""
works

TODO: inputs: X puzzles, try Y times.
1. add wrapper and random selection-from-list functions
to automate so that the promgram generates X number of problem solution sets
2. try Y times and include all novel solutions that do not fail to schedule an event.

"""
import itertools
from typing import List, Set, Tuple

# Define necessary classes

class ScheduleNode:
    """ Represents a node in the scheduling graph (person, place, event, or time). """
    def __init__(self, name: str, node_type: str):
        self.name = name
        self.node_type = node_type
        self.links = set()

    def add_link(self, other_node: 'ScheduleNode'):
        """ Adds a link to another node. """
        self.links.add(other_node)

class EnhancedScheduleNode(ScheduleNode):
    """ Extended node with additional properties for events. """
    def __init__(self, name: str, node_type: str, min_participants: int = 1):
        super().__init__(name, node_type)
        self.min_participants = min_participants


class ScheduleEntity(ScheduleNode):
    """ A specialized node for entities with availability (people, places). """
    def __init__(self, name: str, node_type: str, available_times: Set[str]):
        super().__init__(name, node_type)
        self.available_times = available_times

    def is_available(self, time: str) -> bool:
        """ Checks if the entity is available at a given time. """
        return time in self.available_times

class ConflictCheckingScheduleGraph:
    """ Graph that checks for scheduling conflicts. """
    def __init__(self):
        self.nodes = set()

    def add_node(self, node: ScheduleNode):
        """ Adds a node to the graph. """
        self.nodes.add(node)

    def connect_nodes(self, node1: ScheduleNode, node2: ScheduleNode):
        """ Connects two nodes in the graph. """
        node1.add_link(node2)
        node2.add_link(node1)

    def is_scheduling_possible(self, event: ScheduleNode, time: ScheduleNode, person: ScheduleEntity, place: ScheduleEntity) -> bool:
        """ Checks if scheduling an event is possible without conflicts. """
        # Various checks for availability and existing commitments
        if not person.is_available(time.name) or not place.is_available(time.name):
            return False
        for link in person.links:
            if link.node_type == "Event" and time in link.links:
                return False
        for link in place.links:
            if link.node_type == "Event" and time in link.links:
                return False
        return True

class SchedulingPuzzleGenerator:
    """ Generates scheduling puzzles. """
    def __init__(self, people: List[ScheduleEntity], places: List[ScheduleEntity], events: List[ScheduleNode], times: List[ScheduleNode]):
        self.people = people
        self.places = places
        self.events = events
        self.times = times

    def generate_puzzle(self) -> ConflictCheckingScheduleGraph:
        """ Generates a random scheduling puzzle. """
        puzzle_graph = ConflictCheckingScheduleGraph()
        for entity in self.people + self.places + self.events + self.times:
            puzzle_graph.add_node(entity)
        return puzzle_graph

class ConflictCheckingScheduleGraph:
    """ Graph that checks for scheduling conflicts. """
    def __init__(self):
        self.nodes = set()

    def add_node(self, node: ScheduleNode):
        """ Adds a node to the graph. """
        self.nodes.add(node)

    def connect_nodes(self, node1: ScheduleNode, node2: ScheduleNode):
        """ Connects two nodes in the graph. """
        node1.add_link(node2)
        node2.add_link(node1)

    def is_scheduling_possible(self, event: ScheduleNode, time: ScheduleNode, person: ScheduleEntity, place: ScheduleEntity) -> bool:
        """ Checks if scheduling an event is possible without conflicts. """
        # Various checks for availability and existing commitments
        if not person.is_available(time.name) or not place.is_available(time.name):
            return False
        for link in person.links:
            if link.node_type == "Event" and time in link.links:
                return False
        for link in place.links:
            if link.node_type == "Event" and time in link.links:
                return False
        return True


import random
class SchedulingPuzzleSolver:
    """ Solves scheduling puzzles with enhanced logic for meetings. """

    def solve_puzzle(self, puzzle_graph: ConflictCheckingScheduleGraph, iterations=10) -> List[Tuple[str, str, List[str], str]]:
        best_solution = None
        fewest_unscheduled = float('inf')

        for _ in range(iterations):
            random.shuffle(events)  # Shuffle events to try different orders
            solution, unscheduled = self._attempt_schedule(puzzle_graph)
            if len(unscheduled) < fewest_unscheduled:
                best_solution = solution
                fewest_unscheduled = len(unscheduled)

        if fewest_unscheduled > 0:
            print(f"Could not schedule {fewest_unscheduled} events: {unscheduled}")

        return best_solution

    def _attempt_schedule(self, puzzle_graph: ConflictCheckingScheduleGraph):
        solution = []
        unscheduled = []
        times = [node for node in puzzle_graph.nodes if node.node_type == "Time"]
        people = [node for node in puzzle_graph.nodes if node.node_type == "Person"]
        places = [node for node in puzzle_graph.nodes if node.node_type == "Place"]
        events = [node for node in puzzle_graph.nodes if node.node_type == "Event" or node.node_type == "Meeting"]

        for event in events:
            event_scheduled = False
            if isinstance(event, EnhancedScheduleNode) and event.min_participants > 1:
                # Logic for events that require multiple participants
                for time, place in itertools.product(times, places):
                    available_people = [person for person in people if puzzle_graph.is_scheduling_possible(event, time, person, place)]
                    if len(available_people) >= event.min_participants:
                        for person in available_people[:event.min_participants]:
                            puzzle_graph.connect_nodes(event, time)
                            puzzle_graph.connect_nodes(event, person)
                            puzzle_graph.connect_nodes(event, place)
                        solution.append((event.name, time.name, [p.name for p in available_people[:event.min_participants]], place.name))
                        event_scheduled = True
                        break
            else:
                # Logic for regular events or meetings with only one participant
                for time, person, place in itertools.product(times, people, places):
                    if puzzle_graph.is_scheduling_possible(event, time, person, place):
                        puzzle_graph.connect_nodes(event, time)
                        puzzle_graph.connect_nodes(event, person)
                        puzzle_graph.connect_nodes(event, place)
                        solution.append((event.name, time.name, [person.name], place.name))
                        event_scheduled = True
                        break
            if not event_scheduled:
                unscheduled.append(event.name)

        return solution, unscheduled


def print_puzzle(puzzle_graph: ConflictCheckingScheduleGraph):
    """ Print the puzzle details before solving. """
    print("Puzzle Details:\n")

    # Print availability of people and places
    for node in puzzle_graph.nodes:
        if isinstance(node, ScheduleEntity):
            print(f"{node.name} ({node.node_type}) is available at times: {node.available_times}")

    # Print events to be scheduled
    print("\nEvents to be scheduled:")
    for node in puzzle_graph.nodes:
        if node.node_type in ["Event", "Meeting"]:  # Include both 'Event' and 'Meeting' types
            print(f"- {node.name}")



def print_solution(solution: List[Tuple[str, str, str, str]]):
    """ Prints the solution to the puzzle. """
    print("\nSolution:")
    for event, time, person, place in solution:
        print(f"{event} is scheduled in the {time} with {person} in {place}")

######################
# Generate the puzzle
######################
# Main execution

# Create entities for puzzle generation
people = [ScheduleEntity("Alice", "Person", {"Morning"}), ScheduleEntity("Bob", "Person", {"Afternoon", "Morning"})]
places = [ScheduleEntity("Room A", "Place", {"Morning"}), ScheduleEntity("Room B", "Place", {"Afternoon"})]
events = [
    EnhancedScheduleNode("Team Meeting", "Meeting", min_participants=2),  # A meeting requiring at least 2 participants
    ScheduleNode("Training Session", "Event")  # A regular event
]
times = [ScheduleNode("Morning", "Time"), ScheduleNode("Afternoon", "Time")]

# Initialize the puzzle generator
puzzle_generator = SchedulingPuzzleGenerator(people, places, events, times)

# Generate the puzzle
puzzle_graph = puzzle_generator.generate_puzzle()

# Print the puzzle
print_puzzle(puzzle_graph)

# Solve the puzzle
solver = SchedulingPuzzleSolver()
solution = solver.solve_puzzle(puzzle_graph)

# Print the solution
print_solution(solution)


Puzzle Details:

Room B (Place) is available at times: {'Afternoon'}
Alice (Person) is available at times: {'Morning'}
Room A (Place) is available at times: {'Morning'}
Bob (Person) is available at times: {'Afternoon', 'Morning'}

Events to be scheduled:
- Training Session
- Team Meeting

Solution:
Training Session is scheduled in the Afternoon with ['Bob'] in Room B
Team Meeting is scheduled in the Morning with ['Alice', 'Bob'] in Room A
